In [4]:
import numpy as np
import pandas as pd
from math import factorial

# 방사형 다항식 계산
def radial_polynomial(n, m, r):
    radial = np.zeros_like(r)
    for k in range((n - abs(m)) // 2 + 1):
        coeff = (-1)**k * factorial(n - k) / (
            factorial(k) * factorial((n + abs(m)) // 2 - k) * factorial((n - abs(m)) // 2 - k)
        )
        radial += coeff * r**(n - 2 * k)
    return radial

# Zernike 다항식 계산
def zernike_polynomial(n, m, r, theta):
    if (n - abs(m)) % 2 != 0:
        return np.zeros_like(r)
    radial = radial_polynomial(n, abs(m), r)
    if m > 0:
        return radial * np.cos(m * theta)
    elif m < 0:
        return radial * np.sin(-m * theta)
    else:
        return radial

# Zernike 회귀 분석
def zernike_regression(df, target_col, max_degree=5):
    """
    Perform Zernike regression to fit the target column.
    df: DataFrame with 'r', 'theta', and target column (e.g., 'dx', 'dy')
    target_col: Name of the column to fit (e.g., 'dx', 'dy')
    max_degree: Maximum degree of the Zernike polynomial
    """
    r = df['r'].values
    theta = df['theta'].values
    target = df[target_col].values  # 종속변수

    # Zernike 다항식 생성
    zernike_terms = []
    term_names = []
    for n in range(max_degree + 1):
        for m in range(-n, n + 1, 2):
            term = zernike_polynomial(n, m, r, theta)
            zernike_terms.append(term)
            term_names.append(f"Z_{n}_{m}")
    
    # 디자인 매트릭스 구성
    X = np.vstack(zernike_terms).T

    # 최소자승법으로 계수 계산
    coeffs, _, _, _ = np.linalg.lstsq(X, target, rcond=None)

    # 예측값 계산
    fit = X.dot(coeffs)

    # 잔차 계산
    residuals = target - fit

    # 결과 반환
    results = pd.DataFrame({
        'Zernike_Term': term_names,
        'Coefficient': coeffs
    })
    return results, fit, residuals

# 데이터 준비: .csv 파일 읽기
def prepare_data_from_csv(filepath):
    """
    Load data from a .csv file and prepare it for Zernike regression.
    File must contain 'coordinate_X', 'coordinate_Y', 'dx', 'dy'.
    """
    # .csv 파일 읽기
    df = pd.read_csv(filepath)

    # 필수 열 확인
    required_columns = ['coordinate_X', 'coordinate_Y', 'X_reg', 'Y_reg']
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Input file must contain the columns: {required_columns}")

    # 극좌표 변환
    df['r'] = np.sqrt(df['coordinate_X']**2 + df['coordinate_Y']**2)
    df['theta'] = np.arctan2(df['coordinate_Y'], df['coordinate_X'])

    return df

# 메인 실행
if __name__ == "__main__":
    # 데이터 준비
    filepath = 'RawData-1.csv'  # .csv 파일 경로
    try:
        df = prepare_data_from_csv(filepath)
    except FileNotFoundError:
        print(f"File '{filepath}' not found. Please check the file path.")
        exit()

    # dx 회귀 분석
    zernike_results_dx, fit_dx, residuals_dx = zernike_regression(df, 'X_reg', max_degree=4)

    # dy 회귀 분석
    zernike_results_dy, fit_dy, residuals_dy = zernike_regression(df, 'Y_reg', max_degree=4)

    # 결과 출력
    print("Zernike Coefficients (dx):")
    print(zernike_results_dx.head())
    
    print("\nZernike Coefficients (dy):")
    print(zernike_results_dy.head())

    # 잔차 확인
    print("\nSample Residuals (dx):", residuals_dx[:5])
    print("Sample Residuals (dy):", residuals_dy[:5])

    # 결과 저장
    zernike_results_dx.to_csv('zernike_coefficients_dx.csv', index=False)
    zernike_results_dy.to_csv('zernike_coefficients_dy.csv', index=False)
    print("\nResults saved to 'zernike_coefficients_dx.csv' and 'zernike_coefficients_dy.csv'.")



Zernike Coefficients (dx):
  Zernike_Term   Coefficient
0        Z_0_0 -4.262159e-20
1       Z_1_-1  6.577500e-17
2        Z_1_1 -2.458693e-17
3       Z_2_-2  1.819784e-12
4        Z_2_0  1.194149e-13

Zernike Coefficients (dy):
  Zernike_Term   Coefficient
0        Z_0_0 -3.853747e-20
1       Z_1_-1  2.947575e-17
2        Z_1_1 -1.180165e-17
3       Z_2_-2  5.866292e-13
4        Z_2_0 -2.810385e-12

Sample Residuals (dx): [0.00185889 0.00174789 0.00094689 0.00115889 0.00045889]
Sample Residuals (dy): [-0.00093651 -0.00107751  0.00073349 -0.00026951 -0.00089351]

Results saved to 'zernike_coefficients_dx.csv' and 'zernike_coefficients_dy.csv'.
